In [1]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import ipywidgets

In [2]:
notebook_login()

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("data/tweet_text_2.csv", encoding = 'cp1252') 
tweet_text = data['TWEET_TEXT']
tweet_lengths = [len(tweet) for tweet in tweet_text]


Finetuning BERT to see performance for predicting gasoline shortage

In [19]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=max, truncation=True)
model = AutoModel.from_pretrained(model_name)

tokenizing data and creating trainign an test sets 

In [ ]:
data["tokenized_tweets"] = data['TWEET_TEXT'].apply(lambda x: tokenizer(x))
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data)]

False